In [0]:
# Notebook 3: GOLD - Preparar datos listos para BI incremental

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, min
from delta.tables import DeltaTable

spark = SparkSession.builder.getOrCreate()

SILVER_TABLE = "workspace.javier_altoe.crypto_silver"
GOLD_TABLE = "workspace.javier_altoe.crypto_gold"

df_silver = spark.table(SILVER_TABLE)

# Ejemplo de agregados
df_gold_new = df_silver.groupBy("symbol").agg(
    avg("price_usd").alias("avg_price_usd"),
    max("market_cap").alias("max_market_cap"),
    min("market_cap").alias("min_market_cap")
)

# Si la tabla Gold no existe, crearla
if not spark._jsparkSession.catalog().tableExists(GOLD_TABLE):
    df_gold_new.write.format("delta").saveAsTable(GOLD_TABLE)
else:
    delta_gold = DeltaTable.forName(spark, GOLD_TABLE)
    delta_gold.alias("t").merge(
        df_gold_new.alias("s"),
        "t.symbol = s.symbol"
    ).whenMatchedUpdateAll(
    ).whenNotMatchedInsertAll(
    ).execute()

print("✅ Gold actualizado incrementalmente")
